https://medium.com/swlh/music-genre-classification-using-transfer-learning-pytorch-ea1c23e36eb8
여기 참고

CNN을 이용해서 음악 분류하는 모델 개발

음악 -> 사진 -> cnn -> classify

In [1]:
import torch
import torchvision
import torchaudio
import random
import numpy as np
import librosa
import librosa.display
import pandas as pd
import os
from PIL import Image
import pathlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
from tqdm.autonotebook import tqdm
import IPython.display as ipd
from torchvision.transforms import ToTensor,transforms

set DataPath

In [2]:
data_path = '/csehome/titieiti/data/genres'
img_path = 'img_data'

Now we will convert the music files i.e .wav files to image format using the Librosa library. A detailed explanation can be found [here]('https://www.kaggle.com/msripooja/steps-to-convert-audio-clip-to-spectrogram').


In [3]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in tqdm(genres):
    pathlib.Path(f'{img_path}/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'{data_path}/{g}'):
        songname = f'{data_path}/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'{img_path}/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

  0%|          | 0/10 [00:00<?, ?it/s]

/csehome/titieiti/sinabro/lib/python3.9/site-packages/matplotlib/axes/_axes.py:7580: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


<Figure size 576x576 with 0 Axes>

Visualize

In [ ]:
import matplotlib.image as mpimg

img=mpimg.imread(img_path+'/blues/blues00093.png')
imgplot = plt.imshow(img)
plt.show()
print('shape of image is:',img.shape)

Now we will create a custom dataset:

In [4]:
batch_size = 8
image_size = 224

def normalization_parameter(dataloader):
    mean = 0.
    std = 0.
    nb_samples = len(dataloader.dataset)
    for data,_ in tqdm(dataloader):
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
    mean /= nb_samples
    std /= nb_samples
    return mean.numpy(),std.numpy()

train_transforms = transforms.Compose([transforms.Resize((image_size,image_size)),transforms.ToTensor()])
train_data = torchvision.datasets.ImageFolder(root = img_path, transform = train_transforms)
train_loader =  DataLoader(train_data, batch_size = batch_size , shuffle = True)
mean,std = normalization_parameter(train_loader)

  0%|          | 0/125 [00:00<?, ?it/s]

In [5]:
mean,std

(array([0.8915528 , 0.61322933, 0.5463382 ], dtype=float32),
 array([0.1420371 , 0.33962226, 0.38705954], dtype=float32))

In [6]:
train_transforms = transforms.Compose([transforms.Resize((image_size,image_size)),
                                        transforms.RandomResizedCrop(size=315, scale=(0.95, 1.0)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.CenterCrop(size=299),  # Image net standards
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
test_transforms = transforms.Compose([transforms.Resize((299,299)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)
])
# test_transforms = transforms.Compose([transforms.Resize((image_size,image_size)),
#                                         transforms.RandomResizedCrop(size=315, scale=(0.95, 1.0)),
#                                         transforms.RandomHorizontalFlip(),
#                                         transforms.CenterCrop(size=299),  # Image net standards
#                                         transforms.ToTensor(),
#                                         transforms.Normalize(mean,std)])

In [7]:
#data loader
dataset = torchvision.datasets.ImageFolder(root = img_path, transform =train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = img_path, transform = test_transforms)

In [8]:
#encoder and decoder to convert classes into integer
def encoder(data):
    #label of classes
    classes = data.classes
    encoder = {}
    for i in range(len(classes)):
        encoder[i] = classes[i]
    return encoder

def decoder(data):
    #label of classes
    classes = data.classes
    
    decoder = {}
    for i in range(len(classes)):
        decoder[classes[i]] = i
    return decoder

In [9]:
#plotting rondom images from dataset
def class_plot(data,n_figures = 12):
    n_row = int(n_figures/4)
    fig,axes = plt.subplots(figsize=(14, 10), nrows = n_row, ncols=4)
    for ax in axes.flatten():
        a = random.randint(0,len(data))
        (image,label) = data[a]
#         print(type(image))
        label = int(label)
        encoders = encoder(data)
        l = encoders[label]
        image = image.numpy().transpose(1,2,0)
        im = ax.imshow(image)
        ax.set_title(l)
        ax.axis('off')
    plt.show()

In [ ]:
class_plot(test_dataset)

Split 10% for val, 90% for train

In [13]:
torch.manual_seed(43)
val_size = int(len(dataset)*0.2)
train_size = len(dataset) - val_size

from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [train_size,val_size])
len(train_ds), len(val_ds)

(800, 200)

In [14]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

Preview train dataloader

In [ ]:
for images, _ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

In [17]:
def accuracy(outputs, labels):
    _,preds = torch.max(outputs,dim=1)
    return torch.tensor(torch.sum(preds == labels).item()/len(preds))

In [18]:
class MultilabelImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch
        out = self(images)
        loss = F.cross_entropy(out, targets)
        return loss
    
    def validation_step(self, batch):
        images, targets = batch
        out = self(images)
        loss = F.cross_entropy(out, targets)
        score = 
        (out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epochs_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['val_loss'], result['val_score']))

Model로는 작은 Resnet34 사용

In [149]:
import torchvision.models as models

class Net(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=True)

        # Replace last layer
        num_ftrs = self.network.fc.in_features
        out_features = 30
        self.network.fc = torch.nn.Sequential(OrderedDict([
            ('ff', nn.Linear(num_ftrs, out_features)),
            ('class', nn.Linear(out_features, 10)),
        ]))
    
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self):
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
        
    def unfreeze(self):
        for param in self.network.parameters():
            param.require_grad = True

In [69]:
def evaluate(model,val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [163]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, test_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    test_history = []
    
    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase
        model.train()
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            loss.backward()
            
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step()
        result = evaluate(model, val_loader)
        # test_result = evaluate(model, test_loader)
        result['lrs'] = lrs
        model.epochs_end(epoch, result)
        history.append(result)
        # test_history.append(test_result)
    return {
        'val': history,
        'test': test_history
    }

In [71]:
def plot_losses(history):
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss vs. No. of epochs');
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [72]:
torch.cuda.is_available()

True

In [73]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [74]:
device = get_default_device()
device

device(type='cuda')

In [75]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [76]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [155]:
model = Net()

In [156]:
model = model.to(device)

In [157]:
history = [evaluate(model, val_loader)]
test_history = [evaluate(model, test_loader)]
result = []

In [158]:
[{'val_loss': v['val_loss'], 'val_score': v['val_score']} for v in history]

[{'val_loss': 2.344752788543701, 'val_score': 0.125}]

In [159]:
[{'val_loss': v['val_loss'], 'val_score': v['val_score']} for v in test_history]

[{'val_loss': 2.3447484970092773, 'val_score': 0.1140872985124588}]

In [160]:
epochs = 5
max_lr = 0.001
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [161]:
model.freeze()

In [162]:
%%time
result = fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, test_loader,
                         weight_decay=weight_decay, grad_clip=grad_clip, opt_func=opt_func)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [0], val_loss: 1.9070, val_score: 0.3269


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1], val_loss: 1.8313, val_score: 0.3558


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [2], val_loss: 1.6811, val_score: 0.4087


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [3], val_loss: 1.4497, val_score: 0.4519


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [4], val_loss: 1.3676, val_score: 0.5000
CPU times: user 44.5 s, sys: 11 s, total: 55.5 s
Wall time: 1min 7s


In [164]:
history.extend(result['val'])
test_history.extend(result['test'])

In [165]:
model.unfreeze()

In [166]:
%%time
result = fit_one_cycle(60, max_lr, model, train_loader, val_loader, test_loader,
                         weight_decay=weight_decay, grad_clip=grad_clip, opt_func=opt_func)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [0], val_loss: 1.3645, val_score: 0.5144


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1], val_loss: 1.2558, val_score: 0.5529


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [79], val_loss: 1.3025, val_score: 0.7163


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [80], val_loss: 1.1975, val_score: 0.7356


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [170]:
evaluate(model, train_loader)

{'val_loss': 0.02205146662890911, 'val_score': 0.9925000071525574}

In [171]:
torch.save(model, 'model.pth')

In [172]:
[{'val_loss': v['val_loss'], 'val_score': v['val_score']} for v in history]

[{'val_loss': 2.344752788543701, 'val_score': 0.125},
 {'val_loss': 1.9070414304733276, 'val_score': 0.32692307233810425},
 {'val_loss': 1.8313136100769043, 'val_score': 0.35576921701431274},
 {'val_loss': 1.6810696125030518, 'val_score': 0.4086538553237915},
 {'val_loss': 1.4496642351150513, 'val_score': 0.45192307233810425},
 {'val_loss': 1.3676241636276245, 'val_score': 0.5}]

In [173]:
import copy
newmodel = copy.deepcopy(model).network

In [176]:
fc = newmodel.fc
newmodel.fc = fc.ff

In [ ]:
torch.save(newmodel.state_dict(), 'newmodel.pth')